# DSI-24 EEG Triggering: Code Snippets & Quick Reference

This file provides **ready-to-use** Python functions for sending triggers to the **DSI-24** EEG system, with minimal commentary. For detailed explanations (e.g., **tonic** vs. **phasic** or **immediate** vs. **on-flip** timing), please refer to the **Concepts** document. For a full implementation example (such as a Go/No-Go task in PsychoPy), see the corresponding **Implementation** document.

---

## 1. Basic Serial Setup

Before sending triggers, ensure you have:
1. Installed **pyserial** (`pip install pyserial`) 
2. Identified the **COM port** (e.g., `'COM3'`, `'/dev/ttyUSB0'`, etc.).
3. Initialized and reset the trigger line to **0** at the start.

```python
import serial

# Replace 'COM3' with your actual port (Windows), or '/dev/ttyUSB0' on Linux/macOS
ser = serial.Serial('COM3')
ser.write(str.encode(chr(0)))  # Reset trigger line to 0
print("DSI-24 serial port opened. Trigger line set to 0.")
```




## 2. Phasic (Brief) Triggers

### 2.1 Immediate Phasic Trigger
```python
from psychopy import core

def send_trigger_phasic(ser, trigger_value, dur=0.01):
    """
    Send a brief (phasic) trigger immediately.
    
    ser           : serial.Serial object
    trigger_value : int (e.g., 10)
    dur           : float (seconds to hold the trigger)
    """
    start_time = core.getTime()
    ser.write(str.encode(chr(trigger_value)))
    
    elapsed = core.getTime() - start_time
    remaining = dur - elapsed
    if remaining > 0:
        core.wait(remaining)
    
    # Reset back to 0 after the pulse
    ser.write(str.encode(chr(0)))
```

`Usage Example:`

`send_trigger_phasic(ser, 10, dur=0.02)  # 20 ms pulse of value 10`

### 2.2 On-Flip Phasic Trigger

```python
from psychopy import core

def send_trigger_onflip_phasic(win, ser, trigger_value, dur=0.05):
    """
    Schedule a brief phasic trigger to fire on the next screen flip.
    
    win           : psychopy.visual.Window object
    ser           : serial.Serial object
    trigger_value : int (e.g., 20)
    dur           : float (seconds to hold the trigger)
    """
    def turn_trigger_on():
        ser.write(str.encode(chr(trigger_value)))
        print(f"[PHASIC-ON-FLIP] Trigger {trigger_value} sent.")

    # Schedule for next flip
    win.callOnFlip(turn_trigger_on)
    win.flip()          # Trigger goes high here
    core.wait(dur)      # Hold the trigger
    ser.write(str.encode(chr(0)))
    print("[PHASIC-ON-FLIP] Trigger reset to 0.")
```
`Usage Example:`
`send_trigger_onflip_phasic(win, ser, 20, dur=0.05)`   _50 ms pulse of value 20_

---


## 3. Tonic (Sustained) Triggers

### 3.1 Immediate Tonic Trigger

```python
def send_trigger_tonic(ser, trigger_value):
    """
    Set a sustained (tonic) trigger immediately.
    
    ser           : serial.Serial object
    trigger_value : int (e.g., 5)
    """
    ser.write(str.encode(chr(trigger_value)))
    print(f"[TONIC] Trigger set to {trigger_value}. (Must reset later.)")

```
`Usage Example:`
`send_trigger_tonic(ser, 5)`  _Holds value 5 indefinitely or until reset_trigger_tonic is called._


### 3.2 Reset Tonic Trigger

```python
def reset_trigger_tonic(ser):
    """
    Reset a tonic trigger back to 0.
    
    ser : serial.Serial object
    """
    ser.write(str.encode(chr(0)))
    print("[TONIC] Trigger reset to 0.")

```
`Usage Example:`
`reset_trigger_tonic(ser)`  _Immediately sets the line back to 0._


### 3.3 On_Flip Tonic Trigger

```python
def send_trigger_onflip_tonic(win, ser, trigger_value):
    """
    Schedule a tonic trigger to begin on the next screen flip.
    
    win           : psychopy.visual.Window object
    ser           : serial.Serial object
    trigger_value : int (e.g., 9)
    """
    def turn_trigger_on():
        ser.write(str.encode(chr(trigger_value)))
        print(f"[TONIC-ON-FLIP] Trigger {trigger_value} set on flip (reset manually).")

    # Schedule for next flip
    win.callOnFlip(turn_trigger_on)
    win.flip()
```

`Usage Example:`
`send_trigger_onflip_tonic(win, ser, 9)` _The line goes to 9 on flip and stays high until reset_trigger_tonic(ser) is called._
